# Question 1: Do geographical boundaries affect audience reception of runtimes? Have there been any changing trends?

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [2]:
imdb_title_akas = pd.read_csv('unzippedData/imdb.title.akas.csv')
imdb_title_basics = pd.read_csv('unzippedData/imdb.title.basics.csv')
imdb_title_ratings = pd.read_csv('unzippedData/imdb.title.ratings.csv')

### We need the following information from these tables:
 - imdb_title_akas - tconst, region
 - imdb_title_basics - tconst, primary_title, start_year, runtime_minutes
 - imdb_title_ratings - tconst, averagerating, numvotes  

In [3]:
# Rename "title_id" column in imdb_title_akas as "tconst"- 
# these are the same values across all IMDB tables but are named weirdly.
# This also sets tconst as index, which we'll need to ensure matching tables

imdb_title_akas.rename(columns = {'title_id':'tconst'}, inplace=True) 

In [4]:
# Set "tconst" as index in imdb_title_akas.

imdb_title_akas.set_index('tconst', inplace=True)

In [5]:
# Set "tconst" as index in imdb_title_basics.

imdb_title_basics.set_index('tconst', inplace=True)

In [6]:
# Set "tconst" as index in imdb_title_ratings.

imdb_title_ratings.set_index('tconst', inplace=True)

In [7]:
# Boil a imdb_title_basics down to only the primary_title, start_year, and runtime_minutes columns.

trimmed_basics_df = imdb_title_basics.drop(['genres', 'original_title'], axis=1)

In [8]:
#New combined df of should have title, run time, region.

basics_merge_akas_df = trimmed_basics_df.merge(imdb_title_akas['region'],how='left', left_index=True, right_index=True)


In [9]:
basics_merge_akas_df.head(5)

,primary_title,start_year,runtime_minutes,region
tconst,,,,
tt0063540,Sunghursh,2013,175.0,IN
tt0063540,Sunghursh,2013,175.0,NaN
tt0063540,Sunghursh,2013,175.0,IN
tt0063540,Sunghursh,2013,175.0,IN
tt0063540,Sunghursh,2013,175.0,IN


In [10]:
# Create column 'rating_percent' which is 'averagerating' as a percentage.

imdb_title_ratings['rating_percent'] = (imdb_title_ratings['averagerating'] * 10)

In [11]:
trimmed_imdb_ratings_df = imdb_title_ratings.drop(['averagerating', 'numvotes'], axis=1)

In [12]:
# Merge our new rating percent table into our basics_merge_akas
# table to create our final runtime_comparisons table.

runtimes_df = basics_merge_akas_df.merge(trimmed_imdb_ratings_df, how='left', left_index=True, right_index=True)


In [13]:
runtimes_df.head(10)

,primary_title,start_year,runtime_minutes,region,rating_percent
tconst,,,,,
tt0063540,Sunghursh,2013,175.0,IN,70.0
tt0063540,Sunghursh,2013,175.0,NaN,70.0
tt0063540,Sunghursh,2013,175.0,IN,70.0
tt0063540,Sunghursh,2013,175.0,IN,70.0
tt0063540,Sunghursh,2013,175.0,IN,70.0
tt0066787,One Day Before the Rainy Season,2019,114.0,XWW,72.0
tt0066787,One Day Before the Rainy Season,2019,114.0,IN,72.0
tt0066787,One Day Before the Rainy Season,2019,114.0,NaN,72.0
tt0066787,One Day Before the Rainy Season,2019,114.0,NaN,72.0


In [14]:
# Rename start_year column as release_year in runtimes_df.

runtimes_df.rename(columns = {'start_year':'release_year'}, inplace=True) 

In [15]:
# Rename primary_title column as title in runtimes_df.

runtimes_df.rename(columns = {'primary_title':'title'}, inplace=True)

In [16]:
runtimes_df = runtimes_df[~runtimes_df.index.duplicated(keep='first')]

In [17]:
runtimes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146144 entries, tt0063540 to tt9916754
Data columns (total 5 columns):
title              146144 non-null object
release_year       146144 non-null int64
runtime_minutes    114405 non-null float64
region             107489 non-null object
rating_percent     73856 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 6.7+ MB


In [18]:
# Replace 0 values in runtimes_df['release_year'] with NaN.

runtimes_df.release_year.replace(0, np.NaN)

tconst
tt0063540    2013
tt0066787    2019
tt0069049    2018
tt0069204    2018
tt0100275    2017
             ... 
tt9916538    2019
tt9916622    2015
tt9916706    2013
tt9916730    2017
tt9916754    2013
Name: release_year, Length: 146144, dtype: int64

In [19]:
# Replace 0 values in runtimes_df['runtime_minutes'] with NaN.

runtimes_df.runtime_minutes.replace(0, np.NaN)

tconst
tt0063540    175.0
tt0066787    114.0
tt0069049    122.0
tt0069204      NaN
tt0100275     80.0
             ...  
tt9916538    123.0
tt9916622      NaN
tt9916706      NaN
tt9916730    116.0
tt9916754      NaN
Name: runtime_minutes, Length: 146144, dtype: float64

In [20]:
# Replace 0 values in runtimes_df['rating_percent'] with NaN.

runtimes_df.rating_percent.replace(0, np.NaN)

tconst
tt0063540    70.0
tt0066787    72.0
tt0069049    69.0
tt0069204    61.0
tt0100275    65.0
             ... 
tt9916538     NaN
tt9916622     NaN
tt9916706     NaN
tt9916730     NaN
tt9916754     NaN
Name: rating_percent, Length: 146144, dtype: float64

In [21]:
# Drop rows with any NaN values; all columns are needed for analysis.

runtimes_df = runtimes_df.dropna()

In [22]:
runtimes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52158 entries, tt0063540 to tt9903952
Data columns (total 5 columns):
title              52158 non-null object
release_year       52158 non-null int64
runtime_minutes    52158 non-null float64
region             52158 non-null object
rating_percent     52158 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 2.4+ MB


In [23]:
runtimes_df.release_year.astype(int, inplace=True)

tconst
tt0063540    2013
tt0066787    2019
tt0069049    2018
tt0137204    2017
tt0146592    2010
             ... 
tt9846154    2018
tt9879060    2018
tt9899840    2018
tt9899860    2019
tt9903952    2018
Name: release_year, Length: 52158, dtype: int64

In [24]:
# Remove everything outside of 2010 - 2018.

runtimes_df.drop(runtimes_df[runtimes_df['release_year'] > 2018].index, inplace=True) 

In [25]:
# Remove anything with runtime_minutes under 75 minutes and over 240 minutes.

runtimes_df.drop(runtimes_df[(runtimes_df['runtime_minutes'] < 75) | (runtimes_df['runtime_minutes'] > 240)].index, inplace=True)

In [26]:
runtimes_df.describe()

,release_year,runtime_minutes,rating_percent
count,43427.000000,43427.000000,43427.000000
mean,2013.951804,98.807700,62.321689
std,2.459885,18.757636,14.102408
min,2010.000000,75.000000,10.000000
25%,2012.000000,86.000000,54.000000
50%,2014.000000,94.000000,64.000000
75%,2016.000000,106.000000,72.000000
max,2018.000000,240.000000,100.000000


In [27]:
# Too many unique values in 'region' exist. Create a new column, 'domestic',
# which sets a 1 or 0 value if domestic or foreign.

runtimes_df['domestic'] = np.where(runtimes_df['region'] == "US", 1 , 0)

In [28]:
runtimes_df.describe()

,release_year,runtime_minutes,rating_percent,domestic
count,43427.000000,43427.000000,43427.000000,43427.000000
mean,2013.951804,98.807700,62.321689,0.273148
std,2.459885,18.757636,14.102408,0.445581
min,2010.000000,75.000000,10.000000,0.000000
25%,2012.000000,86.000000,54.000000,0.000000
50%,2014.000000,94.000000,64.000000,0.000000
75%,2016.000000,106.000000,72.000000,1.000000
max,2018.000000,240.000000,100.000000,1.000000


In [29]:
# Bin runtime_minutes to simplify swarmplot.
# df_ages['age_by_decade'] = pd.cut(x=df_ages['age'], bins=[20, 29, 39, 49], labels=['20s', '30s', '40s'])

runtimes_df['runtime_bin'] = pd.cut(x=runtimes_df['runtime_minutes'], bins=[75, 100, 125, 150, 175, 200, 225, 250], include_lowest=True)

In [31]:
runtimes_df.head(10)

,title,release_year,runtime_minutes,region,rating_percent,domestic,runtime_bin
tconst,,,,,,,
tt0063540,Sunghursh,2013,175.0,IN,70.0,0,"(150.0, 175.0]"
tt0069049,The Other Side of the Wind,2018,122.0,VE,69.0,0,"(100.0, 125.0]"
tt0137204,Joe Finds Grace,2017,83.0,CA,81.0,0,"(74.999, 100.0]"
tt0146592,Pál Adrienn,2010,136.0,PL,68.0,0,"(125.0, 150.0]"
tt0162942,Children of the Green Dragon,2010,89.0,PL,69.0,0,"(74.999, 100.0]"
tt0176694,The Tragedy of Man,2011,160.0,HU,78.0,0,"(150.0, 175.0]"
tt0192528,Heaven & Hell,2018,104.0,US,40.0,1,"(100.0, 125.0]"
tt0230212,The Final Journey,2010,120.0,US,88.0,1,"(100.0, 125.0]"
tt0247643,Los pájaros se van con la muerte,2011,110.0,VE,40.0,0,"(100.0, 125.0]"


In [ ]:
sns.swarmplot(x='rating_percent', y='runtime_bin', data=runtimes_df, hue='domestic', dodge=True)

plt.title('Runtime Comparisons')
plt.ylabel('Ratings as Percent')
plt.xlabel('Runtime in Minutes')

In [ ]:
### HEADCANNONS
# imdb_title_basics=imdb_title_basics[~(imdb_title_basics['runtime_minutes']>240) & ~(imdb_title_basics['runtime_minutes']<75)]
#imdb_title_basics = imdb_title_basics.drop_duplicates('primary_title', keep=False)
#imdb_title_basics['primary_title'].value_counts()
#imdb_title_basics.rename(columns={'primary_title':'movie'}, inplace=True)
#imdb_title_basics.head()
#imdb_combined = imdb_title_ratings.join(imdb_title_basics, how='left') #IMDB-title-ratings + IMDB-title-basics
#imdb_combined.head()
# imdb_title_akas - title, region
# imdb_title_basics - primary_title, start_year, runtime_minutes
# imdb_title_ratings - tconst, averagerating, numvotes  
# bo_mojo - title, domestic_gross,foreign_gross, year
# the_numbers - movie, domestic_gross, worldwide_gross, release_year ((the_numbers['release_year']=the_numbers['release_date'].dt.year))
# theMovieDB - title, release_date, vote_average, vote_count